### Meeting Summary: MM (12/10)

| Category | Action Item | Details & References |
| :--- | :--- | :--- |
| **Teaching:**<br>Intro to Ocean. Data Analysis | **Create Python Setup Guide** | Develop an instructional guide or Jupyter Notebook for students on "Setting up Python." |
| | **Convert Module 1** | Translate the first class module from MATLAB to Python. |
| | *Reference* | Use **Bia Villas Bôas'** data GitHub as a primary reference for these teaching materials. |
| **Research / Analysis** | **Compute Setdown** | Calculate setdown on the dataset and analyze how it alters the offset on the nonlinear curve fit. |
| | **Wave Separation** | Apply the **Sheremet and Guza** algorithm to separate incoming vs. outgoing waves. |
| | **Stockdon Comparison** | Calculate the $\chi^2$ of $\sqrt{H_0 L_0}$ (Stockdon) to compare against your current fit. |
| | **Monismith Offset** | Re-evaluate the offset Monismith equation using $H_{s6}$ instead of $H_0$. |

In [6]:
import sys
sys.path.append("..")

#### Setdown 

Becker(2014)

$$ \bar{\eta}_f = \frac{H_f^2k_f}{8sinh(2k_f\bar{h_f})} $$

First, i'd like to write a script that computes setdown from a sensor x array 

In [21]:
## SETDOWN SCRIPT

import pandas as pd
import xarray as xr
import numpy as np

def get_setdownBecker(df: pd.DataFrame, S: xr.Dataset, energy_var: str = 'Seta', depth_col: str = 'h') -> xr.Dataset:
    """
    Calculates wave setdown, significant wave height (Hs), and mean period (Tm) 
    based on spectral data and water depth.

    Parameters:
    -----------
    df : pd.DataFrame
        Raw dataframe containing the water depth series. Must have a DateTime index.
    S : xr.Dataset
        Spectra dataset containing the energy density variable and frequency coordinates.
    energy_var : str, optional
        The name of the energy density variable in `S` (default is 'Seta').
    depth_col : str, optional
        The name of the water depth column in `df` (default is 'h').

    Returns:
    --------
    xr.Dataset
        The original `S` dataset updated with new variables: 'setdown', 'h', 'Hs', and 'Tm'.
    """
    
    # 1. Input Validation
    if depth_col not in df.columns:
        raise ValueError(f"Column '{depth_col}' not found in the provided DataFrame.")
    if energy_var not in S.data_vars:
        raise ValueError(f"Variable '{energy_var}' not found in the provided Dataset.")

    # 2. Interpolate Depth (pandas -> xarray)
    # Convert depth series to xarray DataArray
    h_da = xr.DataArray(
        df[depth_col], 
        coords={'time': df.index},
        dims='time'
    )
    
    # Interpolate depth onto the spectra dataset's time coordinates
    s_h = h_da.interp(time=S.time, method='linear')

    # 3. Calculate Bulk Wave Parameters (Hs and Tm)
    # 0th moment
    m0 = S[energy_var].integrate(coord='frequency')
    # 2nd moment
    m2 = (S[energy_var] * S['frequency']**2).integrate(coord='frequency')
    
    Hs = 4 * np.sqrt(m0)
    # Using zero-crossing period approximation (sqrt(m0/m2))
    Tm = np.sqrt(m0 / m2) 

    # 4. Calculate Wavenumber (k)
    # Internal helper for dispersion relation
    def _get_k_vectorized(T_vals, h_vals, g=9.81):
        """
        Vectorized Newton-Raphson solver for dispersion relation.
        sigma^2 = g * k * tanh(kh)
        """
        sigma = 2 * np.pi / T_vals
        k = sigma**2 / g  # Deep water guess
        
        # 3 iterations is usually sufficient for high precision
        for _ in range(3):
            kh = k * h_vals
            tanh_kh = np.tanh(kh)
            f = (g * k * tanh_kh) - sigma**2
            df_dk = g * (tanh_kh + kh * (1 - tanh_kh**2))
            
            # Avoid division by zero if df_dk is 0 (unlikely in physical range but good practice)
            k = k - f / df_dk
        return k

    k_vals = _get_k_vectorized(Tm.values, s_h.values)
    k = xr.DataArray(k_vals, coords=Tm.coords, name='wavenumber')

    # 5. Calculate Setdown
    # Formula: Setdown = - (Hs^2 * k) / (8 * sinh(2*k*h))
    numerator = - Hs**2 * k
    denominator = 8 * np.sinh(2 * k * s_h)
    
    setdown = numerator / denominator

    # 6. Update and Return Dataset
    # We create a copy to avoid mutating the original dataset implicitly
    S_out = S.copy()
    S_out['setdown'] = setdown
    S_out['h'] = s_h
    S_out['Hs'] = Hs
    S_out['Tm'] = Tm
    
    return S_out

In [ ]:
## 1. load data
    # 1.1. spectra
nperseg = 3600 // 2
df6 = pd.read_parquet('../data_interim/df6.parquet')
S6 = sensor_spectra(df6, nperseg=nperseg)
    # 1.2. bulk params
bulk_df = pd.read_parquet('../data_interim/bulk_df.parquet')

## 2. compute setdown
Sf = get_setdownBecker(df6, S6, energy_var='Seta')

In [9]:
bulk_df

,H0,n,h_tide,L0,H0/L0,H0L0,steepness,Tp,Hs_IG_Nearshore
time,,,,,,,,,
2007-12-14 12:15:01.003921408,1.645071,-0.029936,0.173463,91.624145,0.017955,150.728218,0.017955,14.634210,0.196278
2007-12-14 12:30:01.007838208,1.626725,-0.078732,0.197391,82.591279,0.019696,134.353266,0.019696,13.846214,0.209341
2007-12-14 12:45:01.011755007,1.635449,-0.077717,0.220038,88.654160,0.018448,144.989335,0.018448,14.173290,0.210029
2007-12-14 13:00:01.015671808,1.512499,-0.017358,0.241270,76.812256,0.019691,116.178469,0.019691,13.333391,0.205247
2007-12-14 13:15:01.019588608,1.383515,-0.035062,0.260961,69.346901,0.019951,95.942475,0.019951,13.740518,0.168462
...,...,...,...,...,...,...,...,...,...
2008-01-17 22:15:13.945028607,1.636291,0.074588,-0.382151,140.963989,0.011608,230.658171,0.011608,12.676112,0.256726
2008-01-17 22:30:13.948945407,1.836326,0.065204,-0.398928,136.162355,0.013486,250.038532,0.013486,13.333391,0.227943
2008-01-17 22:45:13.952862207,1.638650,0.045569,-0.414873,128.485516,0.012754,210.542782,0.012754,12.949697,0.248969
